<a href="https://colab.research.google.com/github/saichandrapandraju/TF-Developer/blob/main/3.%20NLP/NLP_Processing_tokenize_pad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download the BBC dataset

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv \
    -O /tmp/bbc-text.csv

--2021-04-11 02:25:20--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.164.176, 172.217.2.112, 142.250.65.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.164.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [application/octet-stream]
Saving to: ‘/tmp/bbc-text.csv’

/tmp/bbc-text.csv   100%[===================>]   4.82M  --.-KB/s    in 0.06s   

2021-04-11 02:25:20 (82.1 MB/s) - ‘/tmp/bbc-text.csv’ saved [5057493/5057493]



In [3]:
#Stopwords list from https://github.com/Yoast/YoastSEO.js/blob/develop/src/config/stopwords.js

stop_words =  [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
# print(stop_words)

Read the dataset

In [15]:
import csv
from tqdm.notebook import tqdm

In [16]:
sentences, labels = [], []

In [17]:
with open('/tmp/bbc-text.csv', 'r') as bbc_file:
  csv_reader = csv.reader(bbc_file, delimiter=',')  ## makes things easier
  next(csv_reader)  ## ignores header
  for row in tqdm(csv_reader):
    labels.append(row[0])
    sentence = row[1]
    for word in stop_words:   ## remove stop words from sentence before appending
      token = ' '+word+' '
      sentence = sentence.replace(token, ' ')    ## replace stop word with space
      sentence = sentence.replace('  ', ' ')     ## remove extra space
    sentences.append(sentence)

In [19]:
assert len(sentences) == len(labels)
print(len(sentences))

2225


Pre-process the dataset

In [22]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [23]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<OOV>')   ## create instance of tokenizer

In [24]:
tokenizer.fit_on_texts(sentences)  ## fit on text data so that it'll create vocabulary

In [25]:
word_index = tokenizer.word_index  ## word-index dict based on sentences in prev step

In [28]:
index_word = tokenizer.index_word  ## index-word dict based on sentences in prev step

In [29]:
assert len(word_index)==len(index_word)
len(word_index)

29714

In [30]:
sequences = tokenizer.texts_to_sequences(sentences)  ## convert all sentences to sequences by replacing each word with its index

In [31]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, padding='post')  ## pad all sequences so that they'll be equal length. By default this will consider length of longest sentence as max_length till which it has to pad

In [33]:
pad_sequences.shape    ## (num_samples, max_length)

(2225, 2442)

In [34]:
pad_sequences[0]

array([  96,  176, 1158, ...,    0,    0,    0], dtype=int32)

processing labels

In [35]:
lab_tokenizer = tf.keras.preprocessing.text.Tokenizer()
lab_tokenizer.fit_on_texts(labels)

In [36]:
lab_index = lab_tokenizer.word_index
lab_index

{'business': 2, 'entertainment': 5, 'politics': 3, 'sport': 1, 'tech': 4}

In [37]:
len(lab_index)

5

In [38]:
lab_sequences = lab_tokenizer.texts_to_sequences(labels)    ## no need to pad b/c each labels belongs to single element in lab_index 

In [42]:
lab_sequences[0]

[4]

In [44]:
len(lab_sequences)   ## num_samples

2225